# <div align='center'> 分类任务测试 </div>

In [5]:
%reload_ext autoreload
%autoreload 2

In [6]:
from k12libs.utils.nb_easy import k12ai_run_project

In [7]:
network = 'base_model'
dataset = 'mnist'

In [8]:
project = k12ai_run_project(debug=True, tb_port=9002,
                            framework='k12cv', task='cls', 
                            network=network, dataset=dataset)

Box(children=(Box(children=(VBox(children=(Tab(children=(VBox(children=(HBox(children=(Text(value='cls', conti…